In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key
import config
from config import api_key


# Create dataframes from csv files
df_2017 = pd.read_csv("Data/PrescribingRate2017.csv", encoding="ISO-8859-1")
df_2016 = pd.read_csv("Data/PrescribingRate2016.csv", encoding="ISO-8859-1")
df_2015 = pd.read_csv("Data/PrescribingRate2015.csv", encoding="ISO-8859-1")
df_uszips = pd.read_csv("Data/uszips.csv", encoding="ISO-8859-1",converters={'zip': '{:0>5}'.format}) #this format adds zeros to zip if >5 digits

#-----------------------------------------------------#
#------------Data Munging/Cleaning/Merging------------#
#-----------------------------------------------------#

# First check that files being merged have same column header names, and change so the column being used to merge is uniform
list(df_2017)

['Year',
 'County',
 'State',
 'State/County FIPS Code',
 'Opioid Prescribing Rater per 100']

In [4]:
list(df_2016)

['Year',
 'County',
 'State',
 'FIPS County Code',
 'Opioid Prescribing Rater per 100']

In [5]:
columns_renamed = {'Year': 'Year',
 'County': 'County',
 'State': 'State',
 'State/County FIPS Code': 'FIPS County Code',
 'Opioid Prescribing Rater per 100': 'Opioid Prescribing Rater per 100'}

df_2017 = df_2017.rename(columns=columns_renamed)
list(df_2017)

['Year',
 'County',
 'State',
 'FIPS County Code',
 'Opioid Prescribing Rater per 100']

In [6]:
merged_df = pd.concat([df_2017, df_2016])

In [7]:
merged_df.count()

Year                                6112
County                              6112
State                               6112
FIPS County Code                    6112
Opioid Prescribing Rater per 100    6098
dtype: int64

In [8]:
df_2017.count()

Year                                2969
County                              2969
State                               2969
FIPS County Code                    2969
Opioid Prescribing Rater per 100    2955
dtype: int64

In [9]:
merged_df.head()

,Year,County,State,FIPS County Code,Opioid Prescribing Rater per 100
0,2017,AUTAUGA,AL,1001,106.6
1,2017,BALDWIN,AL,1003,106.7
2,2017,BARBOUR,AL,1005,90.7
3,2017,BIBB,AL,1007,80.6
4,2017,BLOUNT,AL,1009,48.9


In [10]:
grouped_countycode = merged_df.groupby(['FIPS County Code'])

In [11]:
grouped_countycode_year = merged_df.groupby(['Year','FIPS County Code'])

In [12]:
grouped_countycode_year.count()

County  State  Opioid Prescribing Rater per 100
Year FIPS County Code                                                 
2016 1001                   1      1                                 1
     1003                   1      1                                 1
     1005                   1      1                                 1
     1007                   1      1                                 1
     1009                   1      1                                 1
     1011                   1      1                                 1
     1013                   1      1                                 1
     1015                   1      1                                 1
     1017                   1      1                                 1
     1019                   1      1                                 1
     1021                   1      1                                 1
     1023                   1      1                                 1
     1025                   1      1                                 1
     1027                   1      1                                 1
     1029                   1      1                                 1
     1031                   1      1                                 1
     1033                   1      1                                 1
     1035                   1      1                                 1
     1037                   1      1                                 1
     1039                   1      1                                 1
     1041                   1      1                                 1
     1043                   1      1                                 1
     1045                   1      1                                 1
     1047                   1      1                                 1
     1049                   1      1                                 1
     1051                   1      1                                 1
     1053                   1      1                                 1
     1055                   1      1                                 1
     1057                   1      1                                 1
     1059                   1      1                                 1
...                       ...    ...                               ...
2017 55129                  1      1                                 1
     55131                  1      1                                 1
     55133                  1      1                                 1
     55135                  1      1                                 1
     55137                  1      1                                 1
     55139                  1      1                                 1
     55141                  1      1                                 1
     56001                  1      1                                 1
     56003                  1      1                                 1
     56005                  1      1                                 1
     56007                  1      1                                 1
     56009                  1      1                                 1
     56011                  1      1                                 1
     56013                  1      1                                 1
     56015                  1      1                                 1
     56017                  1      1                                 1
     56019                  1      1                                 1
     56021                  1      1                                 1
     56023                  1      1                                 1
     56025                  1      1                                 1
     56027                  1      1                                 1
     56029                  1      1                                 1
     56031                  1      1                                 1
     56033                  1      1           

In [13]:
# Clean uszips data, and combine it with merged df to create df with both FIPS and ZIP

df_uszips.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,all_county_weights,imprecise,military,timezone
0,00951,18.4284,-66.2566,Toa Baja,PR,Puerto Rico,True,NaN,35,32.5,72137,Toa Baja,{'72137':100},False,False,America/Puerto_Rico
1,96863,21.4484,-157.7623,Mcbh Kaneohe Bay,HI,Hawaii,True,NaN,52,708.1,15003,Honolulu,{'15003':100},False,False,Pacific/Honolulu
2,34141,25.9723,-81.0936,Ochopee,FL,Florida,True,NaN,232,0.1,12021,Collier,"{'12021':97.41,'12087':2.59}",False,False,America/New_York
3,78060,28.4469,-98.1071,Oakville,TX,Texas,True,NaN,37,36.3,48297,Live Oak,{'48297':100},False,False,America/Chicago
4,78144,28.9551,-97.8880,Panna Maria,TX,Texas,True,NaN,45,7.7,48255,Karnes,{'48255':100},False,False,America/Chicago


In [14]:
type(df_uszips['zip'])

pandas.core.series.Series

In [15]:
df_uszips_clean = df_uszips[['zip','city','county_fips','county_name']]

In [16]:
df_uszips_clean.head()

,zip,city,county_fips,county_name
0,00951,Toa Baja,72137,Toa Baja
1,96863,Mcbh Kaneohe Bay,15003,Honolulu
2,34141,Ochopee,12021,Collier
3,78060,Oakville,48297,Live Oak
4,78144,Panna Maria,48255,Karnes


In [17]:
list(merged_df)

['Year',
 'County',
 'State',
 'FIPS County Code',
 'Opioid Prescribing Rater per 100']

In [18]:
# Change column names to match with the merged data on the column we want to merge with

columns_renamed = {'zip': 'Zip Code',
 'city': 'City',
 'county_fips': 'FIPS County Code',
 'county_name': 'County Name'}

df_uszips_clean = df_uszips_clean.rename(columns=columns_renamed)
list(df_uszips_clean)

['Zip Code', 'City', 'FIPS County Code', 'County Name']

In [19]:
# Now merge the Zip Code data with the Merged prescription rate dataframe

zip_merged = pd.merge(merged_df, df_uszips_clean, on="FIPS County Code")

In [20]:
zip_merged.head()

,Year,County,State,FIPS County Code,Opioid Prescribing Rater per 100,Zip Code,City,County Name
0,2017,AUTAUGA,AL,1001,106.6,36006,Billingsley,Autauga
1,2017,AUTAUGA,AL,1001,106.6,36003,Autaugaville,Autauga
2,2017,AUTAUGA,AL,1001,106.6,36091,Verbena,Autauga
3,2017,AUTAUGA,AL,1001,106.6,36067,Prattville,Autauga
4,2017,AUTAUGA,AL,1001,106.6,36051,Marbury,Autauga


In [21]:
zip_merged.count()

Year                                65237
County                              65237
State                               65237
FIPS County Code                    65237
Opioid Prescribing Rater per 100    65158
Zip Code                            65237
City                                65237
County Name                         65237
dtype: int64

In [22]:
zip_merged.head()

,Year,County,State,FIPS County Code,Opioid Prescribing Rater per 100,Zip Code,City,County Name
0,2017,AUTAUGA,AL,1001,106.6,36006,Billingsley,Autauga
1,2017,AUTAUGA,AL,1001,106.6,36003,Autaugaville,Autauga
2,2017,AUTAUGA,AL,1001,106.6,36091,Verbena,Autauga
3,2017,AUTAUGA,AL,1001,106.6,36067,Prattville,Autauga
4,2017,AUTAUGA,AL,1001,106.6,36051,Marbury,Autauga
